In [1]:
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from twilio.rest import Client

In [2]:
# Realizando leitura do arquivo com informações necessárias para implementação da automatização
# Exemplo: senhas, logins, números de telefone

with open('info.txt', 'r') as f:
    # Lendo arquivo txt e transformando em dicionário
    info = eval(f.read())
    
    # Realizando leitura de CEP para posterior entrada no site
    CEP = info['CEP']
    # Número do Whatsapp para qual enviará a mensagem
    number_send = info['telefone']
    
    # Login e senha do site do supermercado
    email_login = info['email_login']
    senha_login = info['senha_login']
    
    # Informações do Twilio para envio da mensagem via Whatsapp
    twilio_number = info['twilio_number']
    twilio_sid = info['twilio_sid']
    twilio_auth = info['twilio_auth']
    
    f.close()

In [3]:
# Acessando site via driver do chrome
driver = webdriver.Chrome()
driver.set_page_load_timeout(10)
driver.get('https://supermercadonow.com/produtos/master-supermercados-consolacao/')
driver.maximize_window()

# Acessando e inserindo informações (CEP, login, senha)
driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[2]/div[1]/div/div[3]/button/span').click()
driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div/input').send_keys(CEP)
time.sleep(2)
driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/button[1]').click()
time.sleep(5)
driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[1]/div[1]/input').send_keys(email_login)
driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[1]/div[2]/input').send_keys(senha_login)

# Acessando informações de disponibilidade de horário
driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/div[2]/div/div/div[2]/button').click()
time.sleep(10)
driver.find_element_by_xpath('/html/body/div[1]/div[5]/div/div[2]/div[2]/div/div[2]/button').click()
time.sleep(5)
resultado = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[1]/div/div[3]/div/div[3]/div[1]/div').text
resultado

'Amanhã - 17h às 19h'

In [4]:
# Enquanto o resultado da busca for 'sem disponibilidade' a automatização fara a busca a cada minuto até que haja horário disponível

while resultado == 'sem disponibilidade':
    print(time.strftime("%H %M").replace(' ', ':'), ' - ', 'Horário não encontrado')
    time.sleep(5)
    driver.refresh()
    time.sleep(5)
    try:
        driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div[2]/div[2]/div/div/button').click()
        time.sleep(5)
        resultado = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[1]/div/div[3]/div/div[3]/div[1]/div/span').text
    except:
        time.sleep(5)
        resultado = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[1]/div/div[3]/div/div[3]/div[1]/div/label').text
    time.sleep(40)

encontrado = time.strftime("%H %M").replace(' ', ':') + ' - ' + 'HORÁRIO DISPONÍVEL !!!! \n'   
print(encontrado*10)

22:35 - HORÁRIO DISPONÍVEL !!!! 
22:35 - HORÁRIO DISPONÍVEL !!!! 
22:35 - HORÁRIO DISPONÍVEL !!!! 
22:35 - HORÁRIO DISPONÍVEL !!!! 
22:35 - HORÁRIO DISPONÍVEL !!!! 
22:35 - HORÁRIO DISPONÍVEL !!!! 
22:35 - HORÁRIO DISPONÍVEL !!!! 
22:35 - HORÁRIO DISPONÍVEL !!!! 
22:35 - HORÁRIO DISPONÍVEL !!!! 
22:35 - HORÁRIO DISPONÍVEL !!!! 



In [6]:
# Enviando mensagem no Whatsapp via Twilio

client = Client(twilio_sid, twilio_auth)

from_whatsapp_number = f'whatsapp:{twilio_number}'
to_whatsapp_number = f'whatsapp:{number_send}'

client.messages.create(body = f'Horário disponível em Master Supermercado. {resultado}.',
                       from_ = from_whatsapp_number,
                       to = to_whatsapp_number);